In [1]:
import atd2022

import torch
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
from sktime.performance_metrics.forecasting import (
    MeanAbsoluteScaledError,
    MeanSquaredError,
)
#import  as util
from atd_CoAtNet.atd_CoAtNet import ATD_CoAtNet
from utils.tools import dotdict
from CoAtNet_Forecaster_wrapper import CoAtNetForecaster

In [2]:
import torch
torch.cuda.is_available()

True

# Full Scale Test

In [4]:
torch.cuda.set_device(2) 

In [9]:
args = dotdict()

args.use_gpu=True
args.lr=0.00001
args.batch_size=40
args.train_epochs=1

In [10]:
lispDL =  CoAtNetForecaster(args)

In [11]:
 # Load data
truth = atd2022.io.read_csv()

# Subset the data for the sake of making this fast.
# Remove if you want to run the example on the entire dataset.
truth = truth.head(108)

# Experiment Parameters
window = 100
num_predict = 4
gap = 0
slide = 1

# Create a dataset `Splitter` object for generating train/test splits
splitter = atd2022.backtest.Splitter(
    truth,
    window,
    num_predict,
    gap,
    slide,
    expanding=True,
) 

In [ ]:
 # Populate a list of models that support the `atd2022.forecasters.Forecaster` protocol
# with which will we generate historical forecasts
models = [
    lispDL,
    atd2022.forecasters.PredictMeanForecaster(),
    atd2022.forecasters.ExponentiallyWeightedMovingAverage(),
]

# Compute historical forecasts for all models
predictions = [
    atd2022.backtest.historical_forecast(model, splitter, verbose=True)
    for model in models
]

<CoAtNet_Forecaster_wrapper.CoAtNetForecaster object at 0x7fb55c568ee0>:   0%|          | 0/5 [00:00<?, ?it/s]

Use GPU: cuda:None
train_loss 22.98277473449707
train_loss 12.026276588439941
train_loss 2.3046748638153076


/scratch/wzong/anaconda3/envs/atd2022/lib/python3.9/site-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([40, 1, 20])) that is different to the input size (torch.Size([40, 20])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


train_loss 1.968843698501587
train_loss 3.844921350479126
train_loss 1.6209901571273804
train_loss 4.82570743560791
train_loss 1.5272374153137207
train_loss 21.40088653564453
train_loss 4.512599945068359
train_loss 13.644489288330078
train_loss 3.06559681892395
train_loss 2.0622076988220215
train_loss 4.728172302246094
train_loss 12.350069046020508
train_loss 3.6689722537994385
train_loss 3.1021406650543213
train_loss 1.9661340713500977
train_loss 3.897282361984253
train_loss 3.708792209625244
train_loss 2.920849323272705
train_loss 1.5915124416351318
train_loss 3.7778642177581787
train_loss 1.4060730934143066
train_loss 5.103010177612305
train_loss 2.6550004482269287
train_loss 2.2984440326690674
train_loss 3.5145363807678223
train_loss 5.039549827575684
train_loss 3.9519314765930176
train_loss 0.03728920966386795
train_loss 2.4112443923950195
train_loss 9.62978744506836
train_loss 1.5858469009399414
train_loss 0.004418368451297283
train_loss 4.019041538238525
train_loss 4.83761882781

/scratch/wzong/LISP-ATD-2022/CoAtNet/atd_CoAtNet/atd_CoAtNet.py:154: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pred = model(torch.tensor(inputs).to(device).float()).cpu().detach().numpy()


Use GPU: cuda:None
train_loss 23.018508911132812
train_loss 12.052367210388184
train_loss 2.3271028995513916
train_loss 1.9865188598632812
train_loss 3.8654768466949463
train_loss 1.6401487588882446
train_loss 4.849228858947754
train_loss 1.5449068546295166
train_loss 21.415515899658203
train_loss 4.5335164070129395
train_loss 13.661408424377441
train_loss 3.086364984512329
train_loss 2.0784974098205566
train_loss 4.755134105682373
train_loss 12.373700141906738
train_loss 3.6871485710144043
train_loss 3.1215147972106934
train_loss 1.9812618494033813
train_loss 3.9118404388427734
train_loss 3.727674961090088
train_loss 2.946011781692505
train_loss 1.6063963174819946
train_loss 3.803231716156006
train_loss 1.423923373222351
train_loss 5.124417304992676
train_loss 2.6764087677001953
train_loss 2.323347330093384
train_loss 3.5311007499694824
train_loss 5.073920249938965
train_loss 3.9712765216827393
train_loss 0.03808068111538887
train_loss 2.4341299533843994
train_loss 9.674440383911133
t

In [ ]:
 metric_functions = [
    MeanAbsoluteScaledError(),
    MeanSquaredError(square_root=True),
]

metrics_df = atd2022.metrics.compute_metrics(
    truth,
    predictions,
    metric_functions,
    models=models,
    train=truth,
    verbose=True,
)

metrics_n = atd2022.metrics.compute_nstep_metrics(
    truth,
    predictions,
    metric_functions,
    models=models,
    train=truth,
    verbose=True,
)
display(metrics_df)
display(metrics_n)
display(atd2022.viz.plot_nstep_metrics(metrics_n))